In [1]:
# Import numpy
import numpy as np

# Import & Init jieba
import jieba
jieba.set_dictionary('datas/dict/dict.txt.big')
jieba.load_userdict('datas/dict/edu_dict.txt')

# Import pandas
import pandas as pd
from pandas import Series, DataFrame

# Import util
import time
import re
import sys
import gc

Building prefix dict from /home/sunset/word_contest/datas/dict/dict.txt.big ...
Loading model from cache /tmp/jieba.u849ecfdca27003d306f39ca004b82b5b.cache
Loading model cost 1.192 seconds.
Prefix dict has been built succesfully.


### Load datasets

#### sample test data

In [2]:
sample = pd.read_csv('datas/sample_test_data.txt')
sample

,id,dialogue,options,answer
0,0,A:你這麼快就知道了,B:全家就是你家\tB:付出不是浪費時間\tB:願意為社會付出的人太少了\tB:我都是一個人...,4
1,1,A:每一支冰塊都不同 水質不同 硬度不同,A:紋路不同\tA:也是你唯一發洩情緒的辦法吧\tA:還記不記得那次你在我家巷口\tA:像拼...,0
2,2,A:這樣沖這麼一大塊網子 要多久時間,B:有更舒適的環境\tB:感覺是一隻很貼心的貓咪\tB:我相信一定可以成功的啦\tB:可以講...,5
3,3,A:兒子啊 都幾點了 你還不睡,B:沒關係啦 我來幫你\tB:我在背書啦 就快背好了\tB:造成誤會 不是很可惜嗎\tB:女...,1
4,4,A:孩子還在念書的時候 看到其他小朋友都有父母 而自己沒有媽媽,A:那孩子就是突然\tA:就是說\tA:禮拜一到學校\tA:媽媽還不會開車\tA:就會問我媽...,4
5,5,A:那牠吐絲這三天三夜\tA:你們不就都要一直在這邊守護著牠囉,B:涂先生\tB:那這些蠶寶寶在這邊吐絲\tB:要吐多久啊\tB:溫度三十多度的話\tB:對...,4
6,6,A:奇怪了 濂僑跑到哪兒去了呢 濂僑 你怎麼啦,B:我可能吃壞肚子了 肚子好痛喔\tB:另外一種比法是 一星期有七天\tB:為什麼不可以 同...,0
7,7,A:是什麼讓玉屏的改變這麼大啊,B:當然是紀老師的魔法囉\tB:都交給這位大力士吧\tB:粗重的交給我就對了\tB:我是八年...,0
8,8,A:曉書啊 你手上拿的 該不是最新的遊戲機吧,B:你不要小看這個扯鈴\t B:互相幫忙一下嘛\tB:一個週末出去走走 沒有關係的啦\tB:...,3
9,9,A:失戀的確會讓人家很難過,A:所以你要縮短認真難過的時間\tA:他走了十三個年頭\tA:獲選之後當然是很開心囉\tA:...,0


In [6]:
# Extract sample test datas
x1 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.dialogue.values]
x2 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.options.values]

# Tokenize
x1 = np.array([list(jieba.cut(' '.join(_))) for _ in x1])
x2 = np.array([[list(jieba.cut(s)) for s in _] for _ in x2])
y = np.array(sample.answer.values)
assert(np.sum([len(_)!=6 for _ in x2]) == 0)

#### PPT Gossiping QA

In [16]:
# Extract PPT Gossiping QA datas
with open('datas/raw/Gossiping-QA-Dataset.txt', 'r') as fi:
    gossip_lines = [line.strip().split('\t') for line in fi]
gossip_x1 = [list(jieba.cut(line[0])) for line in gossip_lines if len(line) == 2]
gossip_x2 = [list(jieba.cut(line[1])) for line in gossip_lines if len(line) == 2]

In [18]:
gossip_x1 = [[w.strip() for w in seg if w.strip()!=''] for seg in gossip_x1]
gossip_x2 = [[w.strip() for w in seg if w.strip()!=''] for seg in gossip_x2]

In [24]:
cnt = {}
next_cnt = {}
for i, (q_seg, r_seg) in enumerate(zip(gossip_x1, gossip_x2)):
    for qw in q_seg:
        if qw not in cnt:
            cnt[qw] = 0
            next_cnt[qw] = {}
        cnt[qw] += 1
        now = next_cnt[qw]
        for rw in r_seg:
            if rw not in now:
                now[rw] = 0
            now[rw] += 1

In [29]:
import json

In [30]:
with open('datas/dict/cnt.json', 'w') as fo:
    json.dump(cnt, fo)

In [31]:
with open('datas/dict/next_cnt.json', 'w') as fo:
    json.dump(next_cnt, fo)